In [30]:
import pandas as pd
x_df = pd.read_csv("Doc//ipodata.csv")
x_df.head(1)

,Code,Name,Date,Price,Share,Amount,Warran,Underwriter,Sector,Subsector,Field,Website,Prospectus,Summary_Prospectus,Alamat,Image,Papan Pencatatan
0,GUNA,PT. Gunanusa Eramandiri Tbk.,09 Jul 2024,Rp150,20%,"5,000,000 lot",0,GR - PANIN SEKURITAS TBK.,Barang Konsumen Primer,Makanan & Minuman,Industri makanan dari kedele dan kacang-kacang...,https://www.gunanusaeramandiri.com,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,Kawasan Industri BIIE Hyundai Jl. Inti I Kav. ...,https://storage.googleapis.com/assets.trading....,Pengembangan


In [31]:
x_df["Price"] = x_df["Price"].str.replace("Rp","").astype(int)
x_df["Share"] = x_df["Share"].str.replace("%","").astype(float)
x_df["Warran"] = x_df['Warran'].apply(lambda x: int(x.split(":")[0])/int(x.split(":")[1]) if x != "0" else int(x) )
x_df["Underwriter"] = x_df["Underwriter"].apply(lambda x:x.split("-")[0])
x_df["Amount"] = x_df["Amount"].str.replace(",","")
x_df["Amount"] = x_df["Amount"].str.replace("lot","")
x_df["Amount"] = x_df["Amount"].apply(lambda x:int(x))

In [49]:
x_df.describe()

,Share,Warran,Price,Amount
count,179.000000,179.000000,179.000000,1.790000e+02
mean,20.528212,0.778265,238.474860,1.897662e+08
std,5.990011,1.388035,328.534802,7.714393e+08
min,2.170000,0.000000,70.000000,1.500000e+06
25%,18.380000,0.000000,100.000000,4.461500e+06
50%,20.000000,0.000000,131.000000,7.500000e+06
75%,24.790000,1.180556,205.000000,1.695000e+07
max,39.760000,10.000000,3080.000000,6.200000e+09


In [32]:
feature_enginering = ["Papan Pencatatan","Share","Underwriter","Warran","Sector","Subsector","Price","Amount"]
x_df=x_df[feature_enginering]

In [33]:
y_df = pd.read_csv("Doc//ipo_gain.csv")
y_df.head(1)

,1 candle gain,2 candle gain,3 candle gain,4 candle gain,5 candle gain,1 candle high,2 candle high,3 candle high,4 candle high,5 candle high
0,34.666667,68.0,109.333333,98.666667,94.666667,34.666667,68.0,109.333333,161.333333,116.0


In [34]:
y_df =y_df.drop(columns = ["1 candle gain","2 candle gain","3 candle gain","4 candle gain",
                            "5 candle gain"],axis = 1)
y_df.describe()

,1 candle high,2 candle high,3 candle high,4 candle high,5 candle high
count,179.000000,179.000000,179.000000,179.000000,179.000000
mean,23.836815,33.287351,35.141429,38.545000,41.596536
std,12.682410,34.533160,50.739880,63.002282,73.578330
min,-10.000000,-34.848485,-42.446043,-44.244604,-50.000000
25%,10.755556,6.547619,0.500000,-1.264280,-0.666667
50%,25.000000,25.454545,19.259259,18.095238,17.968750
75%,34.666667,67.817460,63.583333,54.666667,52.231973
max,35.000000,82.000000,144.000000,204.000000,280.000000


In [ ]:
y_df.describe

In [119]:
df = pd.concat([x_df,y_df],axis=1)

d ={
    "Condition":[],
    "Target":[],
    "Probability":[],
    "Occurance":[]
}

# "Papan Pencatatan","Share","Underwriter","Warran","Sector","Subsector","Price","Amount"
def bayes_prob(variable,target_var,val):
        
    for value in df[variable].unique():
        condition = df[df[variable]==value][["Underwriter",target_var]]
        prob = len(condition[condition[target_var]>=val])/len(condition)

        if len(condition) >=4 and prob>= 0.6:
            d["Condition"].append(f"{variable} = {value}")
            d["Target"].append(f"{target_var} >= {val}")
            d["Probability"].append(prob)
            d["Occurance"].append(len(condition))



for col in ["Underwriter","Papan Pencatatan","Sector","Subsector"]:
    i = 1
    for target in y_df.columns:
        bayes_prob(col,target,y_df[target].max()- (i**2))
        i+=1


In [121]:
d = pd.DataFrame(d)

In [124]:
d.head()

,Condition,Target,Probability,Occurance
0,Underwriter = GR,1 candle high >= 34.0,0.600000,5
1,Underwriter = IF,1 candle high >= 34.0,0.750000,4
2,Underwriter = YJ,1 candle high >= 34.0,0.714286,7
3,Underwriter = HD,1 candle high >= 34.0,0.800000,10
4,Underwriter = SH,1 candle high >= 34.0,0.800000,5


In [125]:
d.to_csv("Doc//bayes_probability.csv",index=False)